# 10 - Story Themes and Custom Renderers

## 🧭 Goal

Understand how ODIBI's story rendering system supports themes and custom output formats.

This notebook will:
- Explore ODIBI's built-in themes (DARK_THEME, LIGHT_THEME, CORPORATE_THEME, MINIMAL_THEME)
- Demonstrate theme customization
- Create custom renderer variations
- Compare output across different themes
- Show how to create a simple custom renderer

**Estimated time:** 30 seconds

---

## 🧱 Core Concepts

**Themes in ODIBI:**
```python
# Apply a theme to a story
story = Story("My Analysis", theme=DARK_THEME)
html = story.render()  # Uses dark theme styling
```

**Custom Renderers:**
```python
# Create a custom text renderer
class TextRenderer:
    def render(self, story):
        return story.to_text()  # Custom output format
```

**When to use different themes:**
- **DARK_THEME**: Modern dashboards, developer tools
- **LIGHT_THEME**: Print-friendly reports, presentations
- **CORPORATE_THEME**: Executive reports, formal documentation
- **MINIMAL_THEME**: Clean, distraction-free analysis

## 🔧 Setup

In [ ]:
# ✅ Environment Setup
import os
from pathlib import Path
import pandas as pd
import time

# Navigate to project root
project_root = Path.cwd().parent if Path.cwd().name == 'walkthroughs' else Path.cwd()
os.chdir(project_root)

# Create artifacts directory
artifacts_dir = Path('walkthroughs/.artifacts/10_themes')
artifacts_dir.mkdir(parents=True, exist_ok=True)

# Import ODIBI story and themes
from odibi.story import Story
from odibi.story.themes import DARK_THEME, LIGHT_THEME, CORPORATE_THEME, MINIMAL_THEME

print("✅ Environment ready")
print(f"📁 Artifacts: {artifacts_dir}")

## ▶️ Run: Explore Built-in Themes

In [ ]:
# Create sample data for all stories
df = pd.DataFrame({
    "Quarter": ["Q1", "Q2", "Q3", "Q4"],
    "Revenue": [120000, 145000, 158000, 172000],
    "Expenses": [85000, 92000, 98000, 105000],
    "Profit": [35000, 53000, 60000, 67000]
})

print("📊 Sample Data for Theme Comparison:")
print(df)
print()

## 🎨 Create: Stories with Different Themes

In [ ]:
# Define themes to test
themes = {
    "dark": DARK_THEME,
    "light": LIGHT_THEME,
    "corporate": CORPORATE_THEME,
    "minimal": MINIMAL_THEME
}

print("🎨 Generating stories with different themes...\n")

for theme_name, theme in themes.items():
    # Create story with theme
    story = Story(f"Quarterly Report ({theme_name.title()} Theme)", theme=theme)
    
    # Add content
    story.add_heading("Executive Summary", level=2)
    story.add_text(f"This report demonstrates the {theme_name.upper()} theme styling for ODIBI stories.")
    
    story.add_heading("Financial Performance", level=2)
    story.add_dataframe(df, caption="Quarterly Financial Data")
    
    story.add_heading("Key Insights", level=2)
    story.add_list([
        "Revenue grew 43% from Q1 to Q4",
        "Profit margins improved throughout the year",
        "Operating expenses remained controlled"
    ])
    
    # Render and save
    html = story.render()
    output_file = artifacts_dir / f"story_{theme_name}.html"
    output_file.write_text(html, encoding='utf-8')
    
    print(f"✅ Generated: story_{theme_name}.html ({len(html):,} bytes)")

print("\n✅ All themed stories generated!")

## 🔍 Inspect: Theme Color Schemes

In [ ]:
# Analyze color schemes in HTML output
print("🔍 Analyzing theme color schemes...\n")

theme_colors = {}

for theme_name in themes.keys():
    html_file = artifacts_dir / f"story_{theme_name}.html"
    html = html_file.read_text(encoding='utf-8')
    
    # Extract color indicators (simplified check)
    has_dark_bg = 'background' in html.lower() and ('#1' in html or '#2' in html or '#0' in html)
    has_light_bg = 'background' in html.lower() and ('#f' in html or '#e' in html or 'white' in html.lower())
    
    theme_colors[theme_name] = {
        "file_size": len(html),
        "has_styling": '<style>' in html or 'style=' in html,
        "dark_elements": has_dark_bg,
        "light_elements": has_light_bg
    }
    
    print(f"📄 {theme_name.upper()} Theme:")
    print(f"   Size: {len(html):,} bytes")
    print(f"   Styling: {'✅' if theme_colors[theme_name]['has_styling'] else '❌'}")
    print()

print("✅ Color scheme analysis complete!")

## 🛠️ Create: Custom Text Renderer

In [ ]:
# Create a simple terminal-friendly text renderer
class TerminalTextRenderer:
    """Simple text renderer for terminal/console output."""
    
    def __init__(self, width=80):
        self.width = width
    
    def render(self, story):
        """Render story as plain text."""
        output = []
        
        # Title
        output.append("=" * self.width)
        output.append(story.title.center(self.width))
        output.append("=" * self.width)
        output.append("")
        
        # Content sections
        for section in story.sections:
            if section.get('type') == 'heading':
                level = section.get('level', 1)
                text = section.get('text', '')
                if level == 2:
                    output.append("")
                    output.append(text.upper())
                    output.append("-" * len(text))
                else:
                    output.append("")
                    output.append(text)
            
            elif section.get('type') == 'text':
                output.append(section.get('content', ''))
            
            elif section.get('type') == 'dataframe':
                df = section.get('data')
                caption = section.get('caption', '')
                if caption:
                    output.append(f"\n[Table: {caption}]")
                output.append(df.to_string())
            
            elif section.get('type') == 'list':
                items = section.get('items', [])
                for item in items:
                    output.append(f"  • {item}")
        
        output.append("")
        output.append("=" * self.width)
        
        return "\n".join(output)

print("✅ Custom TerminalTextRenderer created!")

## 🧪 Test: Custom Renderer

In [ ]:
# Create a story and render with custom renderer
story = Story("Quarterly Report (Custom Text Format)")

story.add_heading("Executive Summary", level=2)
story.add_text("This demonstrates a custom terminal-friendly text renderer.")

story.add_heading("Financial Performance", level=2)
story.add_dataframe(df, caption="Quarterly Financial Data")

story.add_heading("Key Insights", level=2)
story.add_list([
    "Revenue grew 43% from Q1 to Q4",
    "Profit margins improved throughout the year",
    "Operating expenses remained controlled"
])

# Render with custom renderer
renderer = TerminalTextRenderer(width=80)
text_output = renderer.render(story)

# Save to file
output_file = artifacts_dir / "story_custom.txt"
output_file.write_text(text_output, encoding='utf-8')

print("📝 Custom Text Output:")
print()
print(text_output)
print()
print(f"✅ Saved to: {output_file}")

## 📊 Generate: Theme Comparison Report

In [ ]:
# Create comparison markdown document
comparison_lines = [
    "# ODIBI Story Theme Comparison",
    "",
    "## Overview",
    "",
    "This document compares the output of ODIBI's built-in story themes.",
    "",
    "## Themes Tested",
    ""
]

# Add theme details
for theme_name in themes.keys():
    html_file = artifacts_dir / f"story_{theme_name}.html"
    size = html_file.stat().st_size
    
    comparison_lines.append(f"### {theme_name.upper()} Theme")
    comparison_lines.append("")
    comparison_lines.append(f"- **File:** `story_{theme_name}.html`")
    comparison_lines.append(f"- **Size:** {size:,} bytes")
    comparison_lines.append("- **Use Case:** ", end="")
    
    if theme_name == "dark":
        comparison_lines.append("Modern dashboards, developer tools")
    elif theme_name == "light":
        comparison_lines.append("Print-friendly reports, presentations")
    elif theme_name == "corporate":
        comparison_lines.append("Executive reports, formal documentation")
    elif theme_name == "minimal":
        comparison_lines.append("Clean, distraction-free analysis")
    
    comparison_lines.append("")

comparison_lines.extend([
    "## Custom Renderer",
    "",
    "- **File:** `story_custom.txt`",
    f"- **Size:** {(artifacts_dir / 'story_custom.txt').stat().st_size:,} bytes",
    "- **Format:** Terminal-friendly plain text",
    "- **Use Case:** CLI tools, log files, email reports",
    "",
    "## Summary",
    "",
    "ODIBI's theme system allows:",
    "",
    "1. **Consistent styling** across reports",
    "2. **Flexible theming** for different audiences",
    "3. **Custom renderers** for any output format",
    "4. **Easy switching** between themes without code changes",
    "",
    "## Files Generated",
    "",
    "- `story_dark.html` - Dark theme",
    "- `story_light.html` - Light theme",
    "- `story_corporate.html` - Corporate theme",
    "- `story_minimal.html` - Minimal theme",
    "- `story_custom.txt` - Custom text renderer",
    "- `theme_comparison.md` - This document"
])

# Save comparison document
comparison_file = artifacts_dir / "theme_comparison.md"
comparison_file.write_text("\n".join(comparison_lines), encoding='utf-8')

print("📊 Theme Comparison Report Generated!")
print()
print(f"✅ Saved to: {comparison_file}")

## ✅ Self-Check

In [ ]:
start_time = time.time()

try:
    # Check all 6 artifacts exist
    required_files = [
        'story_dark.html',
        'story_light.html',
        'story_corporate.html',
        'story_minimal.html',
        'story_custom.txt',
        'theme_comparison.md'
    ]
    
    print("🔍 Checking artifacts...\n")
    
    for filename in required_files:
        filepath = artifacts_dir / filename
        assert filepath.exists(), f"{filename} not found"
        size = filepath.stat().st_size
        print(f"✅ {filename:30} ({size:,} bytes)")
    
    print()
    
    # Check HTML files have content and styling
    html_files = ['story_dark.html', 'story_light.html', 'story_corporate.html', 'story_minimal.html']
    for html_file in html_files:
        content = (artifacts_dir / html_file).read_text(encoding='utf-8')
        assert len(content) > 500, f"{html_file} seems too small"
        assert 'Quarterly Report' in content, f"{html_file} missing expected content"
    
    # Check custom text output
    custom_text = (artifacts_dir / 'story_custom.txt').read_text(encoding='utf-8')
    assert len(custom_text) > 100, "Custom text output too small"
    assert '===' in custom_text or '---' in custom_text, "Custom text missing formatting"
    
    # Check comparison document
    comparison = (artifacts_dir / 'theme_comparison.md').read_text(encoding='utf-8')
    assert '# ODIBI Story Theme Comparison' in comparison, "Comparison doc missing title"
    assert 'DARK' in comparison, "Comparison doc missing DARK theme"
    
    # Verify HTML files have different sizes (indicating different styling)
    html_sizes = [(artifacts_dir / f).stat().st_size for f in html_files]
    # Note: Themes may produce similar sizes, so we just check they all have content
    assert all(s > 500 for s in html_sizes), "Some HTML files are too small"
    
    # Check runtime
    elapsed = time.time() - start_time
    assert elapsed < 30, f"Runtime {elapsed:.1f}s exceeds 30s budget"
    
    print("🎉 Walkthrough verified successfully!")
    print(f"⏱️  Runtime: {elapsed:.2f}s")
    print(f"📊 Files generated: {len(required_files)}")
    print("✅ All checks passed!")
    
except AssertionError as e:
    print(f"❌ Walkthrough failed: {e}")
    raise
except Exception as e:
    print(f"❌ Unexpected error: {e}")
    raise

## 🧠 Reflection

### What You Learned

1. **Built-in Themes**: ODIBI provides 4 production-ready themes (DARK, LIGHT, CORPORATE, MINIMAL)
2. **Theme Application**: Themes are applied at Story creation time and affect all rendered output
3. **Custom Renderers**: You can create custom renderers for any output format (text, JSON, PDF, etc.)

### Where This Fits in ODIBI

```
Story Pipeline:
Data → Story.add_*() → Story.render(theme) → HTML/Custom Output
                              ↑
                    This notebook explained this part!
```

Themes and renderers are the **presentation layer** of ODIBI stories. They separate data analysis from visual styling.

### Key Insights

- **Separation of Concerns**: Story content is independent of styling
- **Reusability**: One story can be rendered in multiple formats
- **Extensibility**: Custom renderers enable any output format
- **Consistency**: Themes ensure consistent branding across reports

---

## ⏭ Next Steps

**Continue to:** [11_pipeline_validation_and_error_handling.ipynb](11_pipeline_validation_and_error_handling.ipynb)

Learn how ODIBI validates pipelines and handles errors gracefully.

**Deep dive:**
- Read `odibi/story/themes.py` - Theme definitions and customization
- Read `odibi/story/renderers.py` - Built-in HTML renderer implementation
- Read `odibi/story/story.py` - Story class and rendering logic